## Trying Value iteration on Taxi Game Environment from Openai
- Following [this](https://www.kaggle.com/charel/learn-by-example-reinforcement-learning-with-gym#Basic-Q-learning-algorithm) tutorial

#### Explore Taxi-v3 Environment

In [1]:
import gym
import numpy as np
from math import inf
from tqdm import tqdm

# List all available environments in openai-gym
# print(*gym.envs.registry.all(), sep='\n')

In [2]:
# Details about Taxi-v3: https://gym.openai.com/envs/Taxi-v3/

env = gym.make('Taxi-v3')

In [3]:
N_STATES, N_ACTIONS = env.observation_space.n, env.action_space.n
print(f'Taxi-v3 environment with {N_STATES} states, and {N_ACTIONS} actions')

Taxi-v3 environment with 500 states, and 6 actions


In [4]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



### Value Iteration

In [5]:
# if value of any state does not improve more than SIGNIFICAN_IMPROVEMENT
# stop updating
SIGNIFICANT_IMPROVEMENT = 0.01 

# the discount factor
GAMMA = 0.9

In [6]:
def choose_best_action(env, V, cur_state):
  """Given a state, and Values, it returns the action
  which corresponds to maximum future reward"""

  best_reward, best_action = -inf, None

  for action in range(N_ACTIONS):
    env.env.s = cur_state
    next_state, reward, done, info = env.step(action)

    # consider the future discounted reward to decide the
    # best action to take from the given state
    reward = reward + GAMMA * V[next_state]

    if reward > best_reward:
      best_reward, best_action = reward, action
    
  env.env.s = cur_state
  return best_action

In [7]:
V = np.zeros(N_STATES)
pbar, iterations = tqdm(), 0

while True:
  # play games and update values till there is
  # no significant improvement
  improvement = 0
  env.reset()
  total_reward = 0

  for cur_state in range(N_STATES):
    next_action = choose_best_action(env, V, cur_state)

    next_state, reward, done, info = env.step(next_action)
    
    # calculate the improvement in value
    improvement = max(improvement, abs(V[cur_state]-(reward + GAMMA * V[next_state])))

    # update the current state's value
    V[cur_state] = reward + GAMMA * V[next_state]
    total_reward += reward

  # if there is no significant improvement in value over all the
  #  states, stop the value iteration
  if improvement < SIGNIFICANT_IMPROVEMENT:# or iterations > 5:
    break

  iterations += 1
  pbar.update(iterations)
  pbar.set_postfix({'improvement': f'{improvement:.3f}', 'reward': total_reward, 'iterations': iterations})

pbar.close()

820it [00:03, 269.54it/s, improvement=0.012, reward=-416, iterations=40]


### Let us play the game using the value function

In [8]:
cur_state = env.reset()
done, total_reward = False, 0
reward = 0

while not done:
  next_action = choose_best_action(env, V, cur_state)

  cur_state, reward, done, info = env.step(next_action)
  env.render()
  print(f'Reward: {reward}')

  total_reward += reward

print(f'Total Reward: {total_reward} | Done: {done}')

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B:_|
+---------+
  (East)
Reward: -1
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | :_